In [33]:
import pandas as pd
import numpy as np
import re
import emoji 
import time
from tqdm import tqdm
from bs4 import BeautifulSoup

# Scrapping
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

# Error Handling
import socket
import urllib.request
from urllib.request import urlopen
from urllib.parse import quote_plus
from urllib.request import urlretrieve
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException

import warnings
warnings.filterwarnings("ignore")

In [34]:
import os

print("Current working directory : ", os.getcwd())

Current working directory :  C:\git\jh_Minor_Projects\cr_sm


In [40]:
df = pd.read_csv("final_product_info.csv")

# ***Combination of brand name and product name***

In [41]:
brand = list(df.loc[:,"브랜드"])
brand_1 = []
for x in brand:
    x = str(x)
    x=x.replace('nan', '없음')
    brand_1.append(x)

lst = []

for string in brand_1: 
    if "(" in string:
        a = string.index("(")
        b = string.index(")")
    
        lst.append(string[0:a] + string[b+1:len(string)])
    else:
        lst.append(string)

# ======================================================
        
brand_pl_stuffs = []
for i in range(32465):
    if str(df['모델명'][i]) != 'nan':
        brand_pl_stuff = str(lst[i]) + " " + str(df['모델명'][i])
    else:
        brand_pl_stuff = str(lst[i]) + "" + str(df['모델명'][i])
    
    brand_pl_stuff = re.sub('()', '', brand_pl_stuff)
    
    if 'nan' in brand_pl_stuff:
        brand_pl_stuff = str(brand_pl_stuff).replace("nan", '')
    brand_pl_stuffs.append(brand_pl_stuff)

# ======================================================
    
rt = []
for x in brand_pl_stuffs:
    st_sv = []
    if " " in x:
        for i in range(1, len(x.split(" "))):
            st_sv.append([x.split(" ")[0] + " " + x.split(" ")[i]])
    else:
        st_sv.append(x)

    rt.append(st_sv)    

---

# ***Combination of brand name and product features***

In [42]:
title = list(df['title'])
title_1 = []
for i in range(len(df)):
    l = []
    
    # 특수기호 제거     
    se = title[i]
    se = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', se)
    
    # 로마숫자 제거     
    pattern = r"\b(?=[MDCLXVIΙ])M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})([IΙ]X|[IΙ]V|V?[IΙ]{0,3})\b\.?"
    se = re.sub(pattern, '&', se)
    
    '''
    문장의 앞, 뒤, 중간 여백을 제거하기 위한 정규표현과정
    '''
    def get_index_str(STR = ''):
        ar = [x for x in range(len(STR)) if STR[x] != ' ']
        return ar

    # 1) 앞부분의 공백제거
    str_min_ind = min(get_index_str(se))
    if str_min_ind > 0:
        se = se[str_min_ind:]
    else:
        se = se
        
    # 2) 뒷부분의 공백제거
    str_max_ind = max(get_index_str(se))  
    if str_max_ind != len(se) - 1:
        se = se[:str_max_ind + 1] 
        
    # 3) 문자열 중간 부분의 2개 이상의 공백을 하나로 변경  
    se = re.sub(' +',' ', se)
    
    if any(lst[i] in s for s in se.split()):
        terminal = re.sub(lst[i],"", se).split(" ")        
        for vocab in terminal:
            if vocab != '':
                l.append(vocab)
        title_1.append(l)
    else:
        title_1.append(se.split())
        

result_f = []
for j in range(len(df)):
    result_f.append(list(set(title_1[j])))
    
# ======================================================    
    
rt_1 = []
for i in range(len(df)):
    
    bd = lst[i]
    terminal = []
    
    if bd != "없음":
        for pf in result_f[i]:
            terminal.append([bd + " " + pf])
                
    else:
        for nf in range(1, len(result_f[i])):
            terminal.append(result_f[i][0] + " " + result_f[i][nf])
            
    rt_1.append(terminal)    

In [43]:
search_words_df = pd.DataFrame()
search_words_df['rt'] = rt
search_words_df['rt_1'] = rt_1
search_words_df.tail(10)

,rt,rt_1
32455,[드로잉에그],"[[드로잉에그 1EA], [드로잉에그 스톤], [드로잉에그 라운드], [드로잉에그 ..."
32456,"[[베네자임 베네자임], [베네자임 블루밍], [베네자임 카밍], [베네자임 샴푸]...","[[베네자임 헤어], [베네자임 리커버리], [베네자임 블루밍], [베네자임 카밍]..."
32457,[드로잉에그],"[[드로잉에그 1EA], [드로잉에그 브로치], [드로잉에그 라운드커브], [드로잉..."
32458,[드로잉에그],"[[드로잉에그 1EA], [드로잉에그 브로치], [드로잉에그 라운드커브], [드로잉..."
32459,"[[베네자임 베네자임], [베네자임 블루밍], [베네자임 카밍], [베네자임 샴푸]...","[[베네자임 트리트먼트], [베네자임 블루밍], [베네자임 카밍], [베네자임 샴푸..."
32460,"[[W.DRESSROOM 바디미스트], [W.DRESSROOM 스윗블룸], [W.D...","[[W.DRESSROOM 블랑드바디], [W.DRESSROOM 히알루론산], [W...."
32461,[[스칼프메드 레드캡슐바이옴]],"[[스칼프메드 두피케어], [스칼프메드 토닉], [스칼프메드 탈모케어], [스칼프메..."
32462,"[[더레드립 더레드립], [더레드립 ], [더레드립 괄사기]]","[[더레드립 BK], [더레드립 괄사기], [더레드립 얼굴], [더레드립 2개], ..."
32463,"[[스킨구하리 에이티엘], [스킨구하리 릴리프], [스킨구하리 로션], [스킨구하리...","[[스킨구하리 릴리프], [스킨구하리 로션], [스킨구하리 200ml], [스킨구하..."
32464,[[스칼프메드 레드캡슐바이옴]],"[[스칼프메드 두피케어], [스칼프메드 레드캡슐바이옴], [스칼프메드 트리트먼트],..."


In [44]:
sc_wd_final_lst = []

for i in range(len(search_words_df)):
    
    sv_phr = []
    lst = sum([search_words_df.iloc[i,0], search_words_df.iloc[i,1]], [])
    
    for phr in lst:
        '''
        phr: phrase
        '''

        if type(phr) == list:
            phr = phr[0]
            if type(phr) == list:
                phr = phr[0]
        sv_phr.append(phr)
    
    sc_wd_final_lst.append(sv_phr)

In [45]:
sc_wd_final_df = pd.DataFrame()

sc_wd_final_lst = [list(set(x)) for x in sc_wd_final_lst]
sc_wd_final_df['original_title'] = df.loc[:,'title']
sc_wd_final_df['search_words'] = sc_wd_final_lst

sc_wd_final_df.tail(10)

,original_title,search_words
32455,드로잉에그 네일파츠 1EA 라운드 꼬임 스톤 0.8X1.1cm 라운드커브,"[드로잉에그 네일파츠, 드로잉에그 1EA, 드로잉에그 라운드, 드로잉에그 라운드커브..."
32456,베네자임 블루밍 카밍 샴푸 500ml + 리커버리 헤어 밤 150ml,"[베네자임 500ml, 베네자임 샴푸, 베네자임 +, 베네자임 헤어, 베네자임 카밍..."
32457,드로잉에그 네일 파츠 1EA 드롭 브로치 골드 1X1cm 라운드커브,"[드로잉에그 1X1cm, 드로잉에그 네일, 드로잉에그 드롭, 드로잉에그 브로치, 드..."
32458,드로잉에그 네일 파츠 1EA 드롭 브로치 골드 0.9X1.4cm 라운드커브,"[드로잉에그 네일, 드로잉에그 드롭, 드로잉에그 브로치, 드로잉에그 1EA, 드로잉..."
32459,베네자임 블루밍 카밍 샴푸 500ml + 카밍 트리트먼트 500ml,"[베네자임 500ml, 베네자임 샴푸, 베네자임 +, 베네자임 카밍, 베네자임 트리..."
32460,블랑드바디 히알루론산 바디미스트 스윗블룸 100ml,"[W.DRESSROOM 바디미스트, W.DRESSROOM 100ml, W.DRESS..."
32461,스칼프메드 레드캡슐바이옴 두피케어 탈모케어 2종세트 (샴푸 400mlx2 + 토닉 ...,"[스칼프메드 탈모케어, 스칼프메드 400mlx2, 스칼프메드 레드캡슐바이옴, 스칼프..."
32462,더레드립 BK 바디 괄사기 2개 피부 마사지기 도자기 경락 유기 마사지 얼굴 괄사,"[더레드립 도자기, 더레드립 괄사, 더레드립 피부, 더레드립 유기, 더레드립 마사지..."
32463,스킨구하리 / ATL 릴리프 로션 / 200ml,"[스킨구하리 200ml, 스킨구하리 로션, 스킨구하리 ATL, 스킨구하리 에이티엘,..."
32464,스칼프메드 레드캡슐바이옴 두피케어 2종세트 (샴푸 400mlx2 + 트리트먼트 40...,"[스칼프메드 400mlx2, 스칼프메드 레드캡슐바이옴, 스칼프메드 트리트먼트, 스칼..."


---

In [402]:
ta[lst_n_blk]

[0, 1, 3, 5]

In [404]:
ta = [1,2,'',4,'', 7,'']
if '' in ta:
    lst_n_blk = [x for x in range(len(ta)) if ta[x] != '']
    ta = [ta[st] for st in lst_n_blk]
ta

[1, 2, 4, 7]

In [406]:
# i made the definition to scrap website online
def scraping(search_word):
    global fl_brd_lst_n, fl_tlt_lst_n
    
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    print(userAgent)
    
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    options.add_argument(f'user-agent={userAgent}')
    wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
    
    socket.setdefaulttimeout(30)
    
    try:    
        url = 'https://www.glowpick.com/searches/keywords'
        wd.get(url)
        time.sleep(5)

        # 해당 웹페이지 접속 시 팝업창 생성으로 인한 스크레핑 오류를 피하기 위하여 xpath click 코드라인 추가     
        wd.find_element_by_xpath("//*[@id='default-layout']/div/div[1]/span/div/div[2]/div[2]/button[2]").click()

        # 크롤링 할 수 있는 이미지 수 최대 확보를 위한 window 화면크기 최대화     
        wd.maximize_window()
        time.sleep(2)

        # 이미지 검색을 윟나 상품명 입력     
        search_box = wd.find_element_by_xpath("//*[@id='default-layout']/div/div[2]/div/div/div/input")
        search_box.send_keys(search_word)
        search_box.send_keys(Keys.ENTER)

        # 모든 정보를 불러오기 위한 스크롤 다운
        time.sleep(2)
        try:
            while True: 
                def parsing_html_with_class(parsing_info = '.products__product.product'):
                    html = wd.page_source
                    soup = soup = BeautifulSoup(html, "html.parser")
                    interm_r = soup.select(parsing_info)
                    return interm_r

                # ***
                product_list = parsing_html_with_class()

                def reading_product_list(input = product_list):
                    global fl_src_lst 
                    fl_src_lst = []
                    for div1 in input:
                        img_src = div1.img
                        try:
                            img_src_r= img_src['src'].replace("?w=120","?w=500")
                            fl_src_lst.append(img_src_r)
                        except KeyError:
                            fl_src_lst.append("")
                    return fl_src_lst


                element = wd.find_element_by_xpath('/html/body/div/div/div/div/main/div/section/div[2]/div/div[1]/span/div[{}]'.format(len(product_list) - 1))
                wd.execute_script("arguments[0].scrollIntoView(true);", element)
                time.sleep(4)

                product_list_2 = parsing_html_with_class()
#                 print(product_list == product_list_2)

                if product_list == product_list_2:
                    global brd_lst, prd_lst, ind 
                    brd_lst = parsing_html_with_class(parsing_info = '.details__brand__title')
                    prd_lst = parsing_html_with_class(parsing_info = '.details__product')                     
                    
                    reading_product_list(input = product_list)
                    ind  = [x for x in range(len(fl_src_lst)) if fl_src_lst[x] == '']
                    ind_ = [x for x in range(len(fl_src_lst)) if fl_src_lst[x] != '']
                    break

        except:
            fl_src_lst = reading_product_list()  
            brd_lst = parsing_html_with_class(parsing_info = '.details__brand__title')
            prd_lst = parsing_html_with_class(parsing_info = '.details__product')                     
            
        fl_brd_lst_n, fl_tlt_lst_n = ([x.text for x in brd_lst], [x.text for x in prd_lst])
            
        if len(ind) > 0:
            for i in ind:                         
                element = wd.find_element_by_xpath('/html/body/div/div/div/div/main/div/section/div[2]/div/div[1]/span/div[{}]'.format(i-1))
                wd.execute_script("arguments[0].scrollIntoView(true);", element)

                wd.find_element_by_xpath('/html/body/div/div/div/div/main/div/section/div[2]/div/div[1]/span/div[{}]'.format(i+1)).click()
                time.sleep(3)
                html = wd.page_source
                soup = soup = BeautifulSoup(html,"html.parser")
                drop_img_lst = soup.select('.image__img')
                drop_img = drop_img_lst[1]
                img__src = drop_img['src']
                img__src_r = img__src.replace("?w=456","?w=500")
                fl_src_lst[i]=img__src_r
                wd.back()
                time.sleep(5)  

    except NoSuchElementException:
        pass
    except UnboundLocalError:
        pass    
        
# ======================================================
        
# Create DataFrame to save brand, stuff_title written by glowpick
s_info_df = pd.DataFrame(columns=('stuff_name', 'brand'))

def remove_emoji(text = ""): 
    return emoji.get_emoji_regexp().sub(u'', text)

def Saving_scraped_result(iteration_st = 0, iteration_ed = 1): 
    for row_num in tqdm(range(iteration_st, iteration_ed)):
        # Create folder name and corder to save data     
        path = "./"
        query = f"row_{row_num}"

        dir_name = path + query
        os.makedirs(dir_name)
        
        src_sc, tlt_sc, brd_sc = [],[],[]
        src_sm, tlt_sm, brd_sm = [],[],[]
        for sw in sc_wd_final_df.iloc[:,1][row_num]:
            sw = remove_emoji(text = sw)
            time.sleep(0.5)
            
            scraping(sw)
            time.sleep(3)

            src_sc.append(fl_src_lst)
            tlt_sc.append(fl_tlt_lst_n)
            brd_sc.append(fl_brd_lst_n)

            src_sm = sum(src_sc,[])
            tlt_sm = sum(tlt_sc,[])
            brd_sm = sum(brd_sc,[])

        s_info_df.loc[row_num] = [tlt_sm, brd_sm]
        
        if '' in src_sm:
            lst_n_blk = [x for x in range(len(src_sm)) if src_sm[x] != '']
            src_sm, tlt_sm, brd_sm = ([src_sm[st] for st in lst_n_blk], [tlt_sm[st] for st in lst_n_blk], [brd_sm[st] for st in lst_n_blk])         
            
        time.sleep(2)
        
        file_name = []
        for n in range(len(src_sm)):
            if n < 10:
                file_name.append(f'0{n}')
            else:
                file_name.append(f'{n}')
        fullfilename = [os.path.join(dir_name, nm) for nm in file_name]
        [urllib.request.urlretrieve(src, f_name) for src, f_name in zip(src_sm, fullfilename)]             
        time.sleep(3)        

In [407]:
Saving_scraped_result(iteration_st = 100, iteration_ed = 150)

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]



Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1464.0 Safari/537.36


====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\mwith\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\mwith\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36




====== WebDriver manager ======


...

Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36




====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\mwith\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


====== WebDriver manager ======


Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.15 (KHTML, like Gecko) Chrome/24.0.1295.0 Safari/537.15


Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\mwith\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache
 14%|███████████▏                                                                    | 7/50 [53:41<5:29:46, 460.16s/it]


ReadTimeoutError: HTTPConnectionPool(host='127.0.0.1', port=50982): Read timed out. (read timeout=<object object at 0x000001B1166F7F80>)

In [394]:
fl_src_lst

['https://dn5hzapyfrpio.cloudfront.net/home/glowmee/upload/product/20200904/1599188663530.PNG?w=500',
 'https://dn5hzapyfrpio.cloudfront.net/product/a0d/a0de4390-2a3b-11eb-a08a-e36518a7c170.jpeg?w=500']

In [396]:

[urllib.request.urlretrieve(src, f_name) for src, f_name in zip(fl_src_lst, ['000', '111'])]  

[('000', <http.client.HTTPMessage at 0x1b13ce5b448>),
 ('111', <http.client.HTTPMessage at 0x1b141cfac08>)]